# Feature Engineering

En el presente Notebook se continuará con los datos limpios generados en la etapa anterior, y se prepararán para utilizarlos en el modelo de Machine Learning. También se elijirá cual es el modelo adecuado para nuestro caso de análisis.

La etapa de análisis de datos y visualización se puede encontrar en el Notebook [*02 Eduardo_Nunez_Analisis_Datos.ipynb*](https://github.com/EduGatoX/datascience02/blob/main/02%20Eduardo_Nunez_Analisis_Datos.ipynb) alojado en GitHub.

### Importación de librerias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Carga de datos limpios

Los datos alojados en [*videogame_data_clean.csv*](https://github.com/EduGatoX/datascience02/blob/main/videogame_data_clean.csv) ya han sido previamente limpiados, es decir, se imputaron nulos y se eliminaron outliers en la etapa anterior.

In [ ]:
df = pd.read_csv("videogame_data_clean.csv")

In [ ]:
df.info()

### Eliminación de variables

Se eliminan las columnas:

- **Unnamed: 0**: Esta columna se elimina porque proviene del guardado anterior en donde no se indicó a la función que la primera columna era index.
- **Name**: Se eliminará la variable *Name* ya que nos interesa evaluar el desempeño de los videojuegos desde un punto de vista general y no idiosincrático a un juego en particular.

In [ ]:
df = df.drop(columns=["Unnamed: 0", "Name"])

### Transformación de variables

Se desea evaluar un modelo para la variable target **Global_Sales**. De esta forma, las variables *NA_Sales*, *EU_Sales*, *JP_Sales* y *Other_Sales* parecen redudantes ya que **Global_Sales** es la suma de las anteriores. Sin embargo, se procede a transformar estas variables a variables porcentuales respecto de **Global Sales** de forma de que representen la distribución de ventas en esas localidades.

In [ ]:
# Se recupera la variable de interes "Global_Sales"
df["Global_Sales"] = df["NA_Sales"]+df["EU_Sales"]+df["JP_Sales"]+df["Other_Sales"]

In [ ]:
# Se transforman las variables de ventas en forma porcentual
df[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]] = df[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales"]].div(df["Global_Sales"], axis=0)

### One Hot Encoding

Se realiza One Hot Encoding de las variables categoricas.

In [ ]:
df = pd.get_dummies(df, dtype=int)

## Feature Selection

### Selección de variables numéricas

#### Eliminación por coeficiente de correlación

Se realizan dos chequeos:
- Baja correlación con *"target"*: Se evalúan coeficientes de correlación bajos respecto de la variable target **Global_sales**, se considera un bajo coeficiente de correlación a un valor menor a **0.2**. Sin embargo, las variables *NA_Sales*, *EU_Sales*, *JP_Sales* y *Other_Sales* no se tocarán ya que tienen importancia teórica en el negocio.

- Multicolinealidad entre variables: Se eliminan las variables que tengan un coeficiente de correlación mayor a **0.9** entre sí.

Se usará la medida de correlación de **Spearman** ya que puede detectar relaciones no-lineales entre las variables.

In [ ]:
df_numeric = df[["Year", "NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales",
                 "playtime", "metacritic", "rating", "exceptional", "recommended",
                 "meh", "skip"]]

spearman_corr = df_numeric.corr(method="spearman")

high_corr = spearman_corr[(spearman_corr > 0.8) | (spearman_corr < -0.8)]
low_corr = spearman_corr[(spearman_corr < 0.2) & (spearman_corr > -0.2)]

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(high_corr,
            vmin=-1,
            vmax=1,
            cbar=True,
            square=True,
            annot=True,
            fmt=".2f",
            annot_kws={"size": 10},
            cmap="coolwarm",
            ax=ax)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(low_corr,
            vmin=-1,
            vmax=1,
            cbar=True,
            square=True,
            annot=True,
            fmt=".2f",
            annot_kws={"size": 10},
            cmap="coolwarm",
            ax=ax)

De lo anterior es posible concluir que:

- Las variables *Year*, *playtime*, y *rating* tienen baja correlación con la variable **Global_Sales**. Por lo tanto, estas variables serán eliminadas
- Las variables *exceptional*, *recommended*, *meh* y *skip* tienen una alta correlación entre sí. Por lo tanto, existe una relación entre ellas y se realizará reducción de dimensionalidad entre esas variables utilizando el método PCA (Principal Component Analysis).

In [ ]:
df = df.drop(columns=["Year", "playtime", "rating"])

#### Reducción de dimensionalidad (Método PCA)

Del punto anterior se observó que las variables *exceptional*, *recommended*, *meh* y *skip* tienen una alta correlación entre sí. Esto sugiere que existe multicolinealidad entre dichas variables. Por lo tanto, se utilizará el método PCA para detectar el parámetro principal que las define.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

Se identifican las variables a transformar y se realiza el escalado de los datos para ejecutar PCA ya que este método es sensible a la escala de los datos.

In [ ]:
# Se define la variable X como las columnas a transformar
X = df[["exceptional", "recommended", "meh", "skip"]].copy(deep=True)

# PCA es sensitivo a la escala por lo que se debe estandarizar
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled

Se ejecuta el método PCA.

In [ ]:
pca = PCA(n_components=None) # se mantienen todos los componentes
X_pca = pca.fit_transform(X_scaled)

Se evalúa la varianza explicada por las variables.

In [ ]:
explained_variance = pca.explained_variance_ratio_
explained_variance

Se mantienen las variables que explican una varianza acumulada de un **95%**. Se detecta que las dos primeras variables del PCA son capaces de explicar el 95% de la varianza.

In [ ]:
# Varianza acumulada
cumulative_variance = np.cumsum(explained_variance)
print("Varianza acumulada explicada:", cumulative_variance)

In [ ]:
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print("Numero de componentes necesarios para tener una varianza > 95%:", n_components_95)

Se reduce la dimensionalidad utilizando la cantidad de componentes necesarios para mantener una varianza explicada superior a un 95%.

In [ ]:
pca = PCA(n_components=n_components_95)
X_reduced = pca.fit_transform(X_scaled)

Se agregan los componentes al Dataframe original y se eliminan las variables originales.

In [ ]:
df_pca = pd.DataFrame(X_reduced, columns=[f"PC{i+1}" for i in range(n_components_95)])
df = pd.concat([df, df_pca], axis=1)

In [ ]:
df = df.drop(columns=["exceptional", "recommended", "meh", "skip"])

Se evalúa la contribución de componentes originales en la componentes principales.

In [ ]:
print("Contribución de componentes:\n",pca.components_)

Se vuelve a evaluar la matriz de correlacion.

In [ ]:
df_numeric = df[["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "Global_Sales",
                "metacritic", "PC1", "PC2"]]
spearman_corr = df_numeric.corr(method="spearman")

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
sns.heatmap(spearman_corr,
            vmin=-1,
            vmax=1,
            cbar=True,
            square=True,
            annot=True,
            fmt=".2f",
            annot_kws={"size": 10},
            cmap="coolwarm",
            ax=ax)

Finalmente, puede observarse que la variable PC2 tiene una baja correlación con **Global_Sales**, por lo tanto, se eliminará del dataset.

In [ ]:
df = df.drop(columns=["PC2"])

### Selección de variables categóricas

Para seleccionar las variables categóricas se utiliza el test chi-cuadrado para evaluar la independencia entre las variables categóricas y las variable **Global_Sales**. Para realizar esto se realizara un test F ANOVA, utilizando la función de scikit-learn `SelectKBest` con `f_regression` como *scoring function*.

Se evalúan los *f-scores* y los *p-values* para cada categoría y se mantienen aquellas categoría cuyos *p-values* sean menores a **0.05**.

In [ ]:
# Se extraen los nombres de las columnas categóricas
numerical_columns = ["NA_Sales", "EU_Sales", "JP_Sales", "Other_Sales", "metacritic", "PC1"]
categorical_columns = list(set(df.columns) - set(numerical_columns))
len(categorical_columns)


In [ ]:
X = df[categorical_columns]
y = df["Global_Sales"]

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

# Realizar fit para evaluar el score usando la funcion f_regression
anova_selector = SelectKBest(score_func=f_regression, k="all")
anova_scores = anova_selector.fit(X, y)

In [ ]:
f_scores, p_values = f_regression(X, y)

In [ ]:
feature_scores = pd.DataFrame({
    "Feature": X.columns,
    "ANOVA_Score": anova_scores.scores_
}).sort_values(by="ANOVA_Score", ascending=False)
feature_scores["F_Score"] = f_scores
feature_scores["p_values"] = p_values
feature_scores = feature_scores[feature_scores["p_values"] >= 0.05]

Finalmente, se mantendrán sólo las categorías cuyo *p-value* sea menor a **0.05**.

In [ ]:
drop_columns = feature_scores["Feature"].values
df = df.drop(columns=drop_columns)


## Guardado del dataset posterior a la selección de características.

Se almacena el dataset obtenido para modelado de machine learning posterior.

In [ ]:
df.to_csv("videogame_data_model.csv", index=False)